In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, random
# %matplotlib inline

import torch
from torch import nn
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

from importlib import reload 
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)


import torch.backends.cudnn as cudnn
cudnn.benchmark = True  
cudnn.deterministic = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print( device )
# if torch.cuda.is_available():
#     torch.cuda.caching_allocator_delete
#     torch.cuda.empty_cache()  # 释放显存
#     print('Memory Allocated', torch.cuda.memory_allocated() )

xl_01dgree = 0.001745 # 0.1度 
xl_02dgree = 0.003490 # 0.2度 
xl_03dgree = 0.005230 # 0.3度
xl_dgree = xl_03dgree
print( str(xl_dgree) )

cuda:0
0.00523


In [2]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

class Model4A(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 256)   # X.size(1)= 20； 定义256个后续中间层
        self.liner_2 = nn.Linear(256, 128)  # 接着，再定义128个后续中间层
        self.liner_3 = nn.Linear(128, 32)   # 接着，再定义32个后续中间层
        self.liner_4 = nn.Linear(32, 1)     
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x
    
class Model4B(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(18, 128)   # X.size(1)= 20； 定义256个后续中间层
        self.liner_2 = nn.Linear(128, 64)  # 接着，再定义128个后续中间层
        self.liner_3 = nn.Linear(64, 32)   # 接着，再定义32个后续中间层
        self.liner_4 = nn.Linear(32, 1)     
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.relu( self.liner_3(x) )
        x = F.sigmoid( self.liner_4(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(29, 256)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(256, 128)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(128, 64)   # 接着，再定义64个后续中间层
        self.liner_4 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [3]:
model = torch.load('../model/test5-8.pth').to(device)

ts_code, trade_date, pct_change, diff, bar, jx_days_ud60, jx_xl_250, jx_xl_120, jx_xl_60,jx_xl_20, jx_xl_10, jx_xl_5, jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, sz_max_5, sz_max_10, sz_max_20 


In [22]:
""" 手工取数据 + 测试 """
def model_app(X_app):
    # X_app
    Y_app = model(X_app)
    # Y_app
    # Y_app
    Y_app2 = Y_app.to('cpu').detach()
    # Y_app2
    y_app = Y_app2.numpy() 
    acc_num =  np.argwhere(y_app>0.5)
    return Y_app2, acc_num

column_str = 'ts_code, trade_date, pct_change, diff, bar, jx_days_ud60, \
jx_xl_250, jx_xl_120, jx_xl_60,\
jx_xl_20, jx_xl_10, jx_xl_5, \
jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,\
lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, \
sz_max_5, sz_max_10, sz_max_20 '
print(column_str)

sql_new= "SELECT " + column_str +" \
from day_zhibiao \
where trade_date>='20230701' and sz_max_5>8 and \
jx_days_ud60 > 10 and jx_days_ud60 < 60 and \
jx_xl_60>-"+str(xl_dgree)+" and  \
jx_xl_20>0 "
print(sql_new)
df_new = pd.read_sql_query(sql_new, conn)
df_new.shape, df_new.head(2)
# print( df_new[ df_new['sz_max_5']>8 ] )

X_columns = ['pct_change', 
             'diff', 'bar', 
             'jx_days_ud60', 
             'jx_xl_250', 'jx_xl_120','jx_xl_60', 
             'jx_xl_20', 'jx_xl_10', 'jx_xl_5', 
             'jx_zs_5', 'jx_zs_10','jx_zs_20', 'jx_zs_60', 
             'lj_fl_1_3', 'lj_fl_3_10', 'lj_fl_5_20','lj_fl_20_60']
df_app = df_new[X_columns].copy()  
X_app = torch.from_numpy(df_app.values).type(torch.float32).to(device)
X_app.shape, X_app.size()

Y_app2, acc_num = model_app(X_app) 
acc_num

ts_code, trade_date, pct_change, diff, bar, jx_days_ud60, jx_xl_250, jx_xl_120, jx_xl_60,jx_xl_20, jx_xl_10, jx_xl_5, jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, sz_max_5, sz_max_10, sz_max_20 
SELECT ts_code, trade_date, pct_change, diff, bar, jx_days_ud60, jx_xl_250, jx_xl_120, jx_xl_60,jx_xl_20, jx_xl_10, jx_xl_5, jx_zs_5, jx_zs_10, jx_zs_20, jx_zs_60,lj_fl_1_3, lj_fl_3_10, lj_fl_5_20, lj_fl_20_60, sz_max_5, sz_max_10, sz_max_20  from day_zhibiao where trade_date>='20230701' and sz_max_5>8 and jx_days_ud60 > 10 and jx_days_ud60 < 60 and jx_xl_60>-0.00523 and  jx_xl_20>0 


array([], shape=(0, 2), dtype=int64)

array([], shape=(0, 2), dtype=int64)

In [9]:
# Y_app2.numpy() 

In [10]:
np.argwhere( Y_app2.numpy() > 0.3 )

array([], shape=(0, 2), dtype=int64)

In [11]:
x = Y_app2.numpy()
y = x[ (x[:] > 0.25).all(axis=1) ] 
y.size, y

(2,
 array([[0.26246962],
        [0.25760174]], dtype=float32))

In [12]:
y.size, y

(2,
 array([[0.26246962],
        [0.25760174]], dtype=float32))